# AIモデルの評価：コード、人間、モデルベースの採点

このノートブックでは、Claude v3のようなAIモデルの有効性を評価するために広く使用されている3つの手法について詳しく説明します：

1. コードベースの採点
2. 人間による採点
3. モデルベースの採点

各アプローチを例を通して説明し、AIのパフォーマンスを測定する際のそれぞれの利点と制限について検討します。

## コードベースの採点例：感情分析

この例では、映画レビューの感情を肯定的か否定的かに分類するClaudeの能力を評価します。コードを使用して、モデルの出力が期待される感情と一致するかどうかを確認できます。

In [ ]:
# Store the model name and AWS region for later use
MODEL_NAME = "anthropic.claude-3-haiku-20240307-v1:0"
AWS_REGION = "us-west-2"

%store MODEL_NAME
%store AWS_REGION

In [ ]:
# Install the Anthropic package
%pip install anthropic --quiet

In [ ]:
# Import the AnthropicBedrock class and create a client instance
from anthropic import AnthropicBedrock

client = AnthropicBedrock(aws_region=AWS_REGION)

In [ ]:
# Function to build the input prompt for sentiment analysis
def build_input_prompt(review):
    user_content = f"""Classify the sentiment of the following movie review as either 'positive' or 'negative' provide only one of those two choices:
    <review>{review}</review>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "review": "This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.",
        "golden_answer": "positive"
    },
    {
        "review": "I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.",
        "golden_answer": "negative"
    }
]

# Function to get completions from the model
def get_completion(messages):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return message.content[0].text

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["review"])) for item in eval]

# Print the outputs and golden answers
for output, question in zip(outputs, eval):
    print(f"Review: {question['review']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

# Function to grade the completions
def grade_completion(output, golden_answer):
    return output.lower() == golden_answer.lower()

# Grade the completions and print the accuracy
grades = [grade_completion(output, item["golden_answer"]) for output, item in zip(outputs, eval)]
print(f"Accuracy: {sum(grades) / len(grades) * 100}%")

## 人間による採点例：エッセイの採点

エッセイの採点のようないくつかのタスクは、コードだけで評価することが困難です。この場合、人間の採点者がモデルの出力を評価するためのガイドラインを提供できます。

In [ ]:
# Function to build the input prompt for essay generation
def build_input_prompt(topic):
    user_content = f"""Write a short essay discussing the following topic:
    <topic>{topic}</topic>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "topic": "The importance of education in personal development and societal progress",
        "golden_answer": "A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["topic"])) for item in eval]

# Print the outputs and golden answers
for output, item in zip(outputs, eval):
    print(f"Topic: {item['topic']}\n\nGrading Rubric:\n {item['golden_answer']}\n\nModel Output:\n{output}\n")

## モデルベースの採点例

Claudeを使用して、モデルの応答と採点ルーブリックを提供することで、自身の出力を採点することができます。これにより、通常は人間の判断を必要とするタスクの評価を自動化できます。

### 例1：要約

この例では、Claudeを使用して生成した要約の品質を評価します。これは、モデルがより長いテキストから重要な情報を簡潔かつ正確に抽出する能力を評価する必要がある場合に便利です。カバーすべき重要なポイントを概説するルーブリックを提供することで、採点プロセスを自動化し、要約タスクでのモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for summarization
def build_input_prompt(text):
    user_content = f"""Please summarize the main points of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing summary quality
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the quality of the following summary based on this rubric:
    <rubric>{rubric}</rubric>
    <summary>{output}</summary>
    Provide a score from 1-5, where 1 is poor and 5 is excellent."""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "text": "The Magna Carta, signed in 1215, was a pivotal document in English history. It limited the powers of the monarchy and established the principle that everyone, including the king, was subject to the law. This laid the foundation for constitutional governance and the rule of law in England and influenced legal systems worldwide.",
        "golden_answer": "A high-quality summary should concisely capture the key points: 1) The Magna Carta's significance in English history, 2) Its role in limiting monarchical power, 3) Establishing the principle of rule of law, and 4) Its influence on legal systems around the world."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the summary quality score
print(f"Summary quality score: {grades[0]}")

### 例2：ファクトチェック

この例では、Claudeを使用して主張をファクトチェックし、そのファクトチェックの精度を評価します。これは、モデルが正確な情報と不正確な情報を区別する能力を評価する必要がある場合に便利です。正しいファクトチェックでカバーすべき重要なポイントを概説するルーブリックを提供することで、採点プロセスを自動化し、ファクトチェックタスクでのモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for fact-checking
def build_input_prompt(claim):
    user_content = f"""Determine if the following claim is true or false:
    <claim>{claim}</claim>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing fact-check accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Based on the following rubric, assess whether the fact-check is correct:
    <rubric>{rubric}</rubric>
    <fact-check>{output}</fact-check>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "claim": "The Great Wall of China is visible from space.",
        "golden_answer": "A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["claim"])) for item in eval]

grades = []
for output, item in zip(outputs, eval):
    # Print the claim, fact-check, and rubric
    print(f"Claim: {item['claim']}\n")
    print(f"Fact-check: {output}]\n")
    print(f"Rubric: {item['golden_answer']}\n")
    
    # Grade the fact-check
    grader_prompt = build_grader_prompt(output, item["golden_answer"])
    grade = get_completion(grader_prompt)
    grades.append("correct" in grade.lower())

# Print the fact-checking accuracy
accuracy = sum(grades) / len(grades)
print(f"Fact-checking accuracy: {accuracy * 100}%")

### 例3：トーン分析

この例では、Claudeを使用して特定のテキストのトーンを分析し、その分析の精度を評価します。これは、テキストに表現された感情的な内容と態度を識別し解釈するモデルの能力を評価する必要がある場合に便利です。識別すべきトーンの重要な側面を概説するルーブリックを提供することで、採点プロセスを自動化し、トーン分析タスクでのモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for tone analysis
def build_input_prompt(text):
    user_content = f"""Analyze the tone of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": user_content}]

# Function to build the grader prompt for assessing tone analysis accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the accuracy of the following tone analysis based on this rubric:
    <rubric>{rubric}</rubric>
    <tone-analysis>{output}</tone-analysis>"""
    return [{"role": "user", "content": user_content}]

# Define the evaluation data
eval = [
    {
        "text": "I can't believe they canceled the event at the last minute. This is completely unacceptable and unprofessional!",
        "golden_answer": "The tone analysis should identify the text as expressing frustration, anger, and disappointment. Key words like 'can't believe', 'last minute', 'unacceptable', and 'unprofessional' indicate strong negative emotions."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the tone analysis quality
print(f"Tone analysis quality: {grades[0]}")

これらの例は、コードベース、人間、モデルベースの採点が、さまざまなタスクでClaudeのようなAIモデルを評価するためにどのように使用できるかを示しています。評価方法の選択は、タスクの性質と利用可能なリソースによって異なります。モデルベースの採点は、そうでなければ人間の判断を必要とする複雑なタスクの評価を自動化するための有望なアプローチを提供します。